In [ ]:
import ms_analysis as msa

In [ ]:
mc = msa.MSout("ms_out/pH7eH0ms.txt")

In [ ]:
#Write out the microstate  file ordering based on lowest energy in pickle format
ms_orig_lst = [[ms.E, ms.count, ms.state] for  ms in list((mc.microstates.values()))]
ms_orig_lst = sorted(ms_orig_lst, key = lambda x:x[0])


In [ ]:
# This will convert the charge microstate id to charge id.
id_vs_charge = {}
for conf in msa.conformers:
    id_vs_charge[conf.iconf] = conf.crg

def convert_ms_crg(l, d):
    crg_lst =[[y[0], y[1], [convert_ms_crg(x, d) if isinstance(x, list) else d.get(x, x) for x in y[2]]] for y in l]
    return crg_lst

crg_orig_lst = convert_ms_crg(ms_orig_lst, id_vs_charge )


In [ ]:
#  plot the enthalpy distribution and energy count distribution
# lets flatten the list
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
energy_lst_count =np.asarray([a for a,f in zip([x[0] for x in crg_orig_lst], [x[1] for x in crg_orig_lst]) for _ in range(f)])
energy_lst = np.asarray([x[0] for  x in crg_orig_lst])

fig = plt.figure(figsize = (12,6))
fig.suptitle("MQ Frame 144")

plt.subplot(121)
plt.title(' Total Count (MC)')
sns.histplot(energy_lst_count,binwidth=2, kde = True)
plt.xlabel(" Enthalpy (Kcal/Mol)")


plt.subplot(122)
sns.histplot(energy_lst, binwidth=2)
plt.title('No Count from MC')
plt.xlabel(" Enthalpy (Kcal/Mol)")
fig.savefig("enthalpy_dis.png", dpi = 600, bbox_inches = "tight")

In [ ]:
# This will make a free residues list.
import pandas as pd
free_residues = []
for res in mc.free_residues:
    free_residues.append(msa.conformers[res[0]].resid)
ms_free_residues = pd.DataFrame(free_residues,columns = ["Residue"])
ms_free_residues

In [ ]:
# This will make the fixed ionizable residues list.
fixed_resid = []
for conf in msa.conformers:
    if conf.resid not in free_residues:
        if conf.resid[:3] == "NTR" or conf.resid[:3] == "CTR" or conf.resid[:3] == "HIS" or conf.resid[:3] == "LYS" or conf.resid[:3] == "ARG" or conf.resid[:3] == "CYS" or conf.resid[:3] == "ASP" or conf.resid[:3] == "GLU" or conf.resid[:3] == "TYR":
            if conf.resid not in fixed_resid:
                fixed_resid.append(conf.resid)
    
print(fixed_resid[:3])

In [ ]:
# This will read the back ground charge i.e. not appearing in microstate file.
def read_backcrg(fname, ph):
    fixed_resid_vs_crg = {} 
    lines = open(fname).readlines()[:-4]
    headline = lines.pop(0)
    fields = headline.split()
    t_type = fields[0].strip()
    t_points = [float(x) for x in fields[1:]]
    index_ph = t_points.index(float(ph)) + 1
    for line in lines:
        fields = line.split()
        if (fields[0][:3] + fields[0][4:]) in fixed_resid:
            fixed_resid_vs_crg[(fields[0][:3] + fields[0][4:])] = round(float(fields[index_ph]), 0)
    back_crg= round(sum(list(fixed_resid_vs_crg.values())),0)

    return back_crg, fixed_resid_vs_crg

In [ ]:
# make sure give the same ph value you used to read the ms file
fixed_sum_crg = read_backcrg("sum_crg.out", 7)
print(fixed_sum_crg[1])
print(fixed_sum_crg[0])
# THis will make the charge information of fixed residues for MD simulation.
fixed_residues_crg = pd.DataFrame(fixed_sum_crg[1].items(), columns=['Residue', 'crg'])
fixed_residues_crg

In [ ]:
# This will find the unique charge of all charge microstate. Three unique charge state with highest count will select for MD.
crg_all_count = {}
for array in crg_orig_lst:
    if tuple(array[2]) not in crg_all_count.keys():
        crg_all_count[(tuple(array[2]))] = array[1]
    else:
        crg_all_count[(tuple(array[2]))] += array[1]

# make a list of count and charge microstate
all_crg_ms_unique  = []
all_count = []
for u,v in crg_all_count.items():
    all_crg_ms_unique.append(list(u))
    all_count.append(v)
print(f" Total charge ms : {len(crg_orig_lst )}, Total Unique charge ms  {len(all_crg_ms_unique)}")

# make the panda data frame and concatinate
# make panda series and save
import pandas as pd
all_crg_ms_lst = pd.DataFrame(all_crg_ms_unique).T
all_ms_count = pd.DataFrame(all_count,columns = ["Count"]).T
all_crg_ms_count_pd = pd.concat([all_crg_ms_lst.reindex(all_crg_ms_lst.index), all_ms_count])
all_crg_count_res_1 = pd.concat([ms_free_residues, all_crg_ms_count_pd.reindex(all_crg_ms_count_pd.index)], axis=1)
from numpy import inf
all_crg_count_res_1 = all_crg_count_res_1.fillna('Count')
# remove the non titrable residues
all_crg_count_res = all_crg_count_res_1.copy()
for x,y in all_crg_count_res.iterrows():
    if y['Residue'][:3] != "NTR" and y['Residue'][:3] != "CTR" and y['Residue'][:5] != "Count" and y['Residue'][:3] != "ASP" and y['Residue'][:3] != "GLU"\
       and y['Residue'][:3] != "LYS" and y['Residue'][:3] != "TYR" and  y['Residue'][:3] != "ARG" and y['Residue'][:3] != "CYS"and y['Residue'][:3] != "HIS":
        
        all_crg_count_res.drop([x], inplace = True)

all_crg_count_res = all_crg_count_res.set_index("Residue")
# sort based on the count
all_crg_count_res = all_crg_count_res.sort_values(by = "Count", axis = 1, ascending = False)


# take highest three count for MD
all_crg_count_res_md = all_crg_count_res.iloc[:, :3]
all_crg_count_res_md



In [ ]:
# which tautomer charge state is most populated. This includes the background charge also.
import matplotlib.pyplot as plt
import math
%matplotlib inline
x_av = [sum(x) + fixed_sum_crg[0] for x in all_crg_ms_unique]
y_av = [math.log10(x) for x in all_count]
plt.scatter(x_av,y_av)
plt.ylabel("log$_{10}$(Count)")
plt.xlabel("Charge")
plt.xticks(range(int(min(x_av)), int(max(x_av)) + 1))
plt.title("Whole Enthalpy_MQ_fr144")
plt.tight_layout()
plt.savefig("all_en_cr_ph7_vs_log(count).png", dpi = 600, bbox_inches = "tight")



In [ ]:
# find the  10 lowest energy microstate and make unique
low_crg_ms = crg_orig_lst[:10]

low_crg_count = {}
for array in low_crg_ms :
    if tuple(array[2]) not in low_crg_count.keys():
        low_crg_count[(tuple(array[2]))] = array[1]
    else:
        low_crg_count[(tuple(array[2]))] += array[1]

# make a list of count and charge microstate
low_crg_ms_unique = []
low_count = []
for u,v in low_crg_count.items():
    low_crg_ms_unique.append(list(u))
    low_count.append(v)
print(f" Total charge ms : {len(low_crg_ms )}, Total Unique charge ms  {len(low_crg_ms_unique)}")
print(len(low_count))

# make panda series and save
import pandas as pd
low_crg_ms_lst = pd.DataFrame(low_crg_ms_unique).T
low_ms_count = pd.DataFrame(low_count,columns = ["Count"]).T
low_crg_ms_count_pd = pd.concat([low_crg_ms_lst.reindex(low_crg_ms_lst.index), low_ms_count])
low_crg_count_res_1 = pd.concat([ms_free_residues, low_crg_ms_count_pd.reindex(low_crg_ms_count_pd.index)], axis=1)
from numpy import inf
low_crg_count_res_1 = low_crg_count_res_1.fillna('Count')
# remove the non titrable residues
low_crg_count_res = low_crg_count_res_1.copy()
for x,y in low_crg_count_res.iterrows():
    if y['Residue'][:3] != "NTR" and y['Residue'][:5] != "Count" and y['Residue'][:3] != "CTR"and y['Residue'][:3] != "ASP" and y['Residue'][:3] != "GLU"\
       and y['Residue'][:3] != "LYS" and y['Residue'][:3] != "TYR" and  y['Residue'][:3] != "ARG" and y['Residue'][:3] != "CYS"and y['Residue'][:3] != "HIS":
        
        low_crg_count_res.drop([x], inplace = True)

low_crg_count_res = low_crg_count_res.set_index("Residue")
low_crg_count_res

In [ ]:
import matplotlib.pyplot as plt
import math
%matplotlib inline
x2 = [sum(x) + fixed_sum_crg[0] for x in low_crg_ms_unique]
y2 = [math.log10(x) for x in low_count]
plt.scatter(x2,y2)
plt.ylabel("log$_{10}$(Count)")
plt.xlabel("Charge")
plt.xticks(range(int(min(x2)), int(max(x2)) + 1))
plt.title("Min. enthalpy_MQ_fr144")
plt.tight_layout()
plt.savefig("min_en_cr_ph7_vs_log(count).png",bbox_inches = 'tight', dpi = 600)


In [ ]:
# find the charge microstate around +/- 0.0001 Kcal of average energy
av_crg_ms = [[x[0], x[1], x[2]] for x in crg_orig_lst if x[0] >= (mc.average_E  - 0.0001) and  x[0] <= (mc.average_E  + 0.0001)]

if len(av_crg_ms) < 10:
    print("less than ten microstate selected")

av_crg_count = {}
for array in av_crg_ms :
    if tuple(array[2]) not in av_crg_count.keys():
        av_crg_count[(tuple(array[2]))] = array[1]
    else:
        av_crg_count[(tuple(array[2]))] += array[1]

# make a list of count and charge microstate
av_crg_ms_unique = []
av_count = []
for u,v in av_crg_count.items():
    av_crg_ms_unique.append(list(u))
    av_count.append(v)
print(f" Total charge ms : {len(av_crg_ms )}, Total Unique charge ms  {len(av_crg_ms_unique)}")
print(len(av_count)) 

    
# make the panda data frame and concatinate
# make panda series and save
import pandas as pd
av_crg_ms_lst = pd.DataFrame(av_crg_ms_unique).T
av_ms_count = pd.DataFrame(av_count,columns = ["Count"]).T
av_crg_ms_count_pd = pd.concat([av_crg_ms_lst.reindex(av_crg_ms_lst.index), av_ms_count])
av_crg_count_res_1 = pd.concat([ms_free_residues, av_crg_ms_count_pd.reindex(av_crg_ms_count_pd.index)], axis=1)
from numpy import inf
av_crg_count_res_1 = av_crg_count_res_1.fillna('Count')
#remove the non titrable residues
av_crg_count_res = av_crg_count_res_1.copy()
for x,y in av_crg_count_res.iterrows():
    if y['Residue'][:3] != "NTR" and y['Residue'][:3] != "CTR" and y['Residue'][:5] != "Count" and y['Residue'][:3] != "ASP" and y['Residue'][:3] != "GLU"\
       and y['Residue'][:3] != "LYS" and y['Residue'][:3] != "TYR" and  y['Residue'][:3] != "ARG" and y['Residue'][:3] != "CYS"and y['Residue'][:3] != "HIS":
        
        av_crg_count_res.drop([x], inplace = True)

av_crg_count_res = av_crg_count_res.set_index("Residue")
# sort based on Count
av_crg_count_res = av_crg_count_res.sort_values(by = "Count", axis = 1, ascending = False)

# take only three highest count for MD
av_crg_count_res_md = av_crg_count_res.iloc[:, :3]
av_crg_count_res_md

In [ ]:
# Plot of count and tautomer state in avergage charge
import matplotlib.pyplot as plt
import math
%matplotlib inline
x3 = [sum(x) + fixed_sum_crg[0] for x in av_crg_ms_unique]
y3 = [math.log10(x) for x in av_count]
plt.scatter(x3,y3)
plt.ylabel("log$_{10}$(Count)")
plt.xlabel("Charge")
plt.xticks(range(int(min(x3)), int(max(x3)) + 1))
plt.title("Average enthalpy_MQ_fr144 +/- 0.0001")
plt.tight_layout()
plt.savefig("ave_en_cr_ph7_vs_log(count).png", bbox_inches = 'tight', dpi = 600)


In [ ]:
# save all three data frames and non titrable residues charge information.
writer = pd.ExcelWriter('charge_ms_mq144_dry.xlsx', engine='xlsxwriter')
#all_crg_count_res.to_excel(writer, sheet_name='all_crg_count_res')
low_crg_count_res.to_excel(writer, sheet_name='low_crg_count_res')
av_crg_count_res.to_excel(writer, sheet_name='av_crg_count_res')
av_crg_count_res_md.to_excel(writer, sheet_name='av_crg_count_res_md_3')
all_crg_count_res_md.to_excel(writer, sheet_name='all_crg_count_res_md_3')
fixed_residues_crg.to_excel(writer, sheet_name = 'fixed_residues_crg')
writer.save()


In [ ]:
# due to colmn size limit here I have save in csv file
#all_crg_count_res.to_csv('all_crg_count_res.csv',index=False, header = True)
all_crg_count_res.to_csv('all_crg_count_res.csv', header = True)

In [3]:
# Lets read the all_crg file that already save
import pandas as pd
all_crg_count_order = pd.read_csv("all_crg_count_res.csv", index_col = 0) #index_col = 0 means it will not take excel index

all_crg_count_order.shape


(164, 79993)

In [ ]:
all_crg_count_order.head()

In [4]:
cls1 = ["HISE0038", "TYRE0087", "ASPE0139", "ARGE0217", "ARGQ0216", "GLUQ0223", "GLUQ0225"]
cls2 = ["GLUE0050", "GLUE0051", "GLUE0216", "HISP0060", "ARGQ0154", "ASPQ0220", "GLUQ0227", "ARGQ0301", "TYRQ0302", "ASPQ0303", "ARGQ0307"]
con1_2 = ["GLUE0388", "ASPE0392", "ARGQ0299"]
cls3 = ["ARGE0042", "HISE0058", "TYRE0061", "GLUP0045", "ASPP0049", "GLUP0053", "ASPQ0071", "LYSQ0146", "TYRQ0147", "GLUQ0235", "TYRQ0236", "LYSQ0240"]
cls4 = ["ASPP0072", "GLUP0074", "TYRP0109", "GLUP0110", "LYSP0113", "ARGP0117", "TYRQ0124", "GLUQ0130", "TYRQ0134", "TYRQ0162", "GLUQ0163", "TYRQ0206", "GLUQ0213", "HISQ0251", "GLUR0158"]
con1_4 = ["GLUQ0035", "TYRQ0232", "HISQ0233", "GLUQ0248", "TYRQ0249", "ARGQ0294"]
cls5 = ["TYRR0043", "TYRR0059", "GLUS0032"]
cls6 = ["GLUP0006", "TYRP0007", "TYRQ0005", "ASPQ0008", "TYRQ0010", "LYSQ0016", "ASPQ0119", "ASPQ0184", "LYSQ0190", "GLUR0005", "ASPR0047", "ARGR0049", "ASPR0118", "LYSR0120", "TYRS0003", "ARGS0047"]
all_crg_count_order_re1 = all_crg_count_order.copy()
for i,j in all_crg_count_order.iterrows():
    if i[:-1] in cls1: 
        all_crg_count_order_re1.rename(index = {i: i +'cl1'}, inplace = True) 
    if i[:-1] in cls2: 
        all_crg_count_order_re1.rename(index = {i: i +'cl2'}, inplace = True)
    if i[:-1] in con1_2: 
        all_crg_count_order_re1.rename(index = {i: i +'co1_2'}, inplace = True)
    if i[:-1] in cls3: 
        all_crg_count_order_re1.rename(index = {i: i +'cl3'}, inplace = True)
    if i[:-1] in cls4: 
        all_crg_count_order_re1.rename(index = {i: i +'cl4'}, inplace = True)
    if i[:-1] in con1_4: 
        all_crg_count_order_re1.rename(index = {i: i +'co1_4'}, inplace = True)
    if i[:-1] in cls5: 
        all_crg_count_order_re1.rename(index = {i: i +'cl5'}, inplace = True)
    if i[:-1] in cls6: 
        all_crg_count_order_re1.rename(index = {i: i +'cl6'}, inplace = True)
        
all_crg_count_order_re2 = all_crg_count_order_re1.T
all_crg_count_order_re2.head()

Residue,NTRE0026_,HISE0034_,HISE0038_cl1,GLUE0050_cl2,GLUE0051_cl2,GLUE0054_,HISE0058_cl3,HISE0063_,GLUE0071_,HISE0072_,...,ASPS0052_,GLUS0067_,HISS0081_,ARGS0082_,GLUS0083_,ASPS0088_,ASPS0089_,GLUS0092_,CTRS0095_,Count
14,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,24416.0
30,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,20715.0
228,1.0,1.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13483.0
120,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13370.0
345,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,12166.0


In [5]:
# lets find the cluster residues which are present in the ms state
#to see which residues are in the data set from each clusters
ms_cl1 = []
ms_cl2 = []
ms_co1_2 = []
ms_cl3 = []
ms_cl4 = []
ms_co1_4 = []
ms_cl5 = []
ms_cl6 = []

for i in list(all_crg_count_order_re2.columns):
    if i[9:] == 'cl1':
        ms_cl1.append(i)
    if i[9:] == 'cl2':
        ms_cl2.append(i) 
    if i[9:] == 'co1_2':
        ms_co1_2.append(i)   
    if i[9:] == 'cl3':
        ms_cl3.append(i)
    if i[9:] == 'cl4':
        ms_cl4.append(i)       
    if i[9:] == 'co1_4':
        ms_co1_4.append(i)       
    if i[9:] == 'cl5':
        ms_cl5.append(i)    
    if i[9:] == 'cl6':
        ms_cl6.append(i)
print(f'ms_cl1: {ms_cl1}, ms_cl2 =  {ms_cl2}, ms_co1_2 =  {ms_co1_2}, ms_cl3 =  {ms_cl3}, ms_cl4: {ms_cl4}, ms_co1_4: {ms_co1_4}, ms_cl5 =  {ms_cl5}, ms_cl6 =  {ms_cl6}')
    

ms_cl1: ['HISE0038_cl1', 'ASPE0139_cl1', 'GLUQ0223_cl1', 'GLUQ0225_cl1'], ms_cl2 =  ['GLUE0050_cl2', 'GLUE0051_cl2', 'GLUE0216_cl2', 'HISP0060_cl2', 'ARGQ0154_cl2', 'ASPQ0220_cl2', 'GLUQ0227_cl2', 'ARGQ0301_cl2', 'TYRQ0302_cl2', 'ASPQ0303_cl2'], ms_co1_2 =  ['GLUE0388_co1_2', 'ARGQ0299_co1_2'], ms_cl3 =  ['HISE0058_cl3', 'GLUP0053_cl3', 'LYSQ0146_cl3', 'TYRQ0147_cl3', 'GLUQ0235_cl3'], ms_cl4: ['ASPP0072_cl4', 'GLUP0074_cl4', 'GLUP0110_cl4', 'ARGP0117_cl4', 'TYRQ0124_cl4', 'GLUQ0130_cl4', 'GLUQ0163_cl4', 'TYRQ0206_cl4', 'GLUQ0213_cl4', 'HISQ0251_cl4', 'GLUR0158_cl4'], ms_co1_4: ['GLUQ0035_co1_4', 'HISQ0233_co1_4', 'TYRQ0249_co1_4', 'ARGQ0294_co1_4'], ms_cl5 =  ['TYRR0059_cl5', 'GLUS0032_cl5'], ms_cl6 =  ['TYRP0007_cl6', 'ASPQ0008_cl6', 'LYSQ0016_cl6', 'ASPQ0119_cl6', 'LYSQ0190_cl6', 'GLUR0005_cl6', 'ARGR0049_cl6', 'ASPR0118_cl6', 'LYSR0120_cl6', 'ARGS0047_cl6']


In [6]:
#print(fixed_sum_crg[1]). This will make a sum of charge that are not appearing in microstate.
non_ms_cl1 = 0
non_ms_cl2 = 0
non_ms_co1_2 = 0
non_ms_cl3 = 0
non_ms_cl4 = 0
non_ms_co1_4 = 0
non_ms_cl5 = 0
non_ms_cl6 = 0
non_cls_no_ms_crg = 0
list1 = []
list2 = []

for i, j in fixed_sum_crg[1].items():
    if i[:8] in cls1:
        non_ms_cl1 +=  fixed_sum_crg[1][i]
    if i[:8] in cls2:
        non_ms_cl2 +=  fixed_sum_crg[1][i]
    if i[:8] in con1_2:
        non_ms_co1_2 +=  fixed_sum_crg[1][i]
    if i[:8] in cls3:
        non_ms_cl3 +=  fixed_sum_crg[1][i]
    if i[:8] in cls4:
        non_ms_cl4 +=  fixed_sum_crg[1][i]
    if i[:8] in con1_4:
        non_ms_co1_4 +=  fixed_sum_crg[1][i]
    if i[:8] in cls5:
        non_ms_cl5 +=  fixed_sum_crg[1][i]
    if i[:8] in cls6:
        non_ms_cl6 +=  fixed_sum_crg[1][i]
    if i[:8] not in cls1 and i[:8] not in  cls2 and i[:8] not in con1_2 and i[:8] not in cls3 and i[:8] not in cls4 and i[:8] not in con1_4 and i[:8] not in cls5 and i[:8] not in cls6:
        non_cls_no_ms_crg +=  fixed_sum_crg[1][i]
        
    

# print(non_ms_cl1)
# print(non_ms_cl2)
# print(non_ms_co1_2)
# print(non_ms_cl3)
# print(non_ms_cl4)
# print(non_ms_co1_4)
# print(non_ms_cl5)
# print(non_ms_cl6)
# print(non_cls_no_ms_crg)


NameError: name 'fixed_sum_crg' is not defined

In [ ]:
# sanity check the residues
print((non_ms_cl1 + non_ms_cl2 + non_ms_co1_2 + non_ms_cl3 + non_ms_cl4 + non_ms_co1_4 + non_ms_cl5 + non_ms_cl6 + non_cls_no_ms_crg) == fixed_sum_crg[0])


In [ ]:
# save the cluster not appearing sum of residues charges
name_non_ms = ["non_ms_cl1", "non_ms_cl2", "non_ms_co1_2", "non_ms_cl3", "non_ms_cl4", "non_ms_co1_4", "non_ms_cl5", "non_ms_cl6", "non_cls_no_ms_crg"]
charge_non_ms = [non_ms_cl1, non_ms_cl2, non_ms_co1_2, non_ms_cl3, non_ms_cl4, non_ms_co1_4, non_ms_cl5, non_ms_cl6, non_cls_no_ms_crg]
non_ms_crg = pd.DataFrame(list(zip(name_non_ms,charge_non_ms)), columns = ["non ms", "Charge"])
non_ms_crg.to_excel("non_ms_crg.xlsx")

In [ ]:
# here we also need to add the residues that are not appearing in the cluster so
all_crg_count_order_re3 = all_crg_count_order_re2.copy()

all_crg_count_order_re3['Crg_cl1'] =  all_crg_count_order_re3[ms_cl1].sum(axis = 1) + non_ms_cl1
all_crg_count_order_re3['Crg_cl2'] =  all_crg_count_order_re3[ms_cl2].sum(axis = 1) + non_ms_cl2
all_crg_count_order_re3['Crg_co1_2'] =  all_crg_count_order_re3[ms_co1_2].sum(axis = 1) + non_ms_co1_2
all_crg_count_order_re3['Crg_cl3'] = all_crg_count_order_re3[ms_cl3].sum(axis = 1) + non_ms_cl3
all_crg_count_order_re3['Crg_cl4'] = all_crg_count_order_re3[ms_cl4].sum(axis = 1) + non_ms_cl4
all_crg_count_order_re3['Crg_co1_4'] = all_crg_count_order_re3[ms_co1_4].sum(axis = 1) + non_ms_co1_4
all_crg_count_order_re3['Crg_cl5'] =  all_crg_count_order_re3[ms_cl5].sum(axis = 1) + non_ms_cl5
all_crg_count_order_re3['Crg_cl6'] =  all_crg_count_order_re3[ms_cl6].sum(axis = 1) + non_ms_cl6

all_crg_count_order_re3


In [ ]:
# save the data frame that has charge, cluster charge, Count
all_crg_count_order_re3.to_csv("finalall_crg_cl_crg_count_order.csv")

In [7]:
# Now read the charge file that save
import pandas as pd
data1 = pd.read_csv("finalall_crg_cl_crg_count_order.csv",index_col = 0)

In [ ]:
data1.shape
data1

In [ ]:
# This is to define the custom color for plot
custom_palette = {-4: 'r', -3: 'blue', -2: 'pink', -1: 'orange', 0: 'black', 1: 'pink', 2: 'magenta', 3: 'gold' } # k black
# This is to acess the column name
col_gra_list = ['Crg_cl1', 'Crg_cl2', 'Crg_co1_2', 'Crg_cl3', 'Crg_cl4', 'Crg_co1_4', 'Crg_cl5', 'Crg_cl6']
list(enumerate(col_gra_list))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [ ]:
# This wiil make the plot of the count of each charge distribution including count from the mc simulation.create the subplots
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

fig = plt.figure(figsize = (10,5))


fig.suptitle("MQ Frame 144", y= 1.02,fontweight='bold', fontsize = 15)
# fig.text(0.5, 0.04, 'Charge', ha='center')
# fig.text(0.02, 0.5, 'Count', va='center', rotation='vertical')



plt.subplot(241)
#plt.title("Cluster 1")
cl1_data = data1.groupby('Crg_cl1').Count.sum().reset_index()
sns.barplot(x='Crg_cl1', y='Count', data = cl1_data, palette=custom_palette)
#plt.xlabel("Cl.1 Charge")
plt.xlabel("Charge Cl1")
plt.ylabel("Count")


plt.subplot(242)
#plt.title("Cluster 2")
cl2_data = data1.groupby('Crg_cl2').Count.sum().reset_index()
sns.barplot(x='Crg_cl2', y='Count', data = cl2_data, palette=custom_palette)
#plt.xlabel("Cluster 2 charge")
plt.xlabel("Charge Cl2")
plt.ylabel(None)

plt.subplot(243)
#plt.title("Con. 1-2 ")
co1_2_data = data1.groupby('Crg_co1_2').Count.sum().reset_index()
sns.barplot(x='Crg_co1_2', y='Count', data = co1_2_data, palette=custom_palette)
#plt.xlabel("Con. 1-2 charge")
plt.xlabel("Charge Con1_2")
plt.ylabel(None)

plt.subplot(244)
#plt.title("Cluster 3 ")
cl3_data = data1.groupby('Crg_cl3').Count.sum().reset_index()
sns.barplot(x='Crg_cl3', y='Count', data = cl3_data ,palette=custom_palette)
#plt.xlabel("Cluster 3 charge")
plt.xlabel("Charge Cl3")
plt.ylabel(None)


plt.subplot(245)
#plt.title("Cluster 4 ")
cl4_data = data1.groupby('Crg_cl4').Count.sum().reset_index()
sns.barplot(x='Crg_cl4', y='Count', data = cl4_data ,palette=custom_palette)
#plt.xlabel("Cluster 4 charge")
plt.xlabel("Charge Cl4")
plt.ylabel("Count")


plt.subplot(246)
#plt.title("Con.1-4")
Co1_4_data = data1.groupby('Crg_co1_4').Count.sum().reset_index()
sns.barplot(x='Crg_co1_4', y='Count', data = Co1_4_data ,palette=custom_palette)
#plt.xlabel("Con. 1-4 charge")
plt.xlabel("Charge Con1_4")
plt.ylabel(None)

plt.subplot(247)
#plt.title("Cluster 5 ")
cl5_data = data1.groupby('Crg_cl5').Count.sum().reset_index()
sns.barplot(x='Crg_cl5', y='Count', data = cl5_data ,palette=custom_palette)
#plt.xlabel("Cluster 5 charge")
plt.xlabel("Charge Cl5")
plt.ylabel(None)


plt.subplot(248)
#plt.title("Cluster 6 ")
cl6_data = data1.groupby('Crg_cl6').Count.sum().reset_index()
sns.barplot(x='Crg_cl6', y='Count', data = cl6_data ,palette=custom_palette)
#plt.xlabel("Cluster 6 charge")
plt.xlabel("Charge Cl6")
plt.ylabel(None)


plt.tight_layout()
fig.savefig("cluster_charge_count_mq144.png", bbox_inches='tight', dpi = 600)   


In [ ]:
# cluster 4 figure
fig = plt.figure(figsize = (8,4))
plt.title("MQ Frame 144 ")
cl4_data = data1.groupby('Crg_cl4').Count.sum().reset_index()
sns.barplot(x='Crg_cl4', y='Count', data = cl4_data ,palette=custom_palette)
#plt.xlabel("Cluster 4 charge")
plt.xlabel("Charge Cl4")
plt.ylabel("Count");
fig.savefig("Cluster 4_mq_fr144.png", bbox_inches='tight', dpi = 600)



In [ ]:
cl1_data

In [ ]:
# plot the count plot for tatomeric state.This plot will not count the MC count from montecarlo.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize = (30,25))
fig.suptitle("MQ Frame 144", y= 0.90, fontsize = 20)
# define the color

for i in enumerate(col_gra_list):
    
    plt.subplot(4, 4,i[0]+1)
    sns.countplot(x = i[1], data = data1, palette=custom_palette)
    plt.xticks(fontsize = 15)
    plt.yticks(fontsize = 15)
    plt.ylabel("Count",fontsize = 20)
    plt.xlabel(i[1], fontsize = 20)
    plt.yticks(rotation = 45)
    
    
fig.show()

fig.savefig("no_count_cluster_charge_count_mq144.png",bbox_inches='tight', dpi = 300)   


In [8]:
# remove cluster sum  crg information
data2 = data1.iloc[:, :-8]
data2

,NTRE0026_,HISE0034_,HISE0038_cl1,GLUE0050_cl2,GLUE0051_cl2,GLUE0054_,HISE0058_cl3,HISE0063_,GLUE0071_,HISE0072_,...,ASPS0052_,GLUS0067_,HISS0081_,ARGS0082_,GLUS0083_,ASPS0088_,ASPS0089_,GLUS0092_,CTRS0095_,Count
14,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,24416.0
30,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,20715.0
228,1.0,1.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13483.0
120,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,13370.0
345,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,1.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,12166.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77714,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,1.0,-1.0,1.0,...,-1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0
65729,0.0,1.0,1.0,0.0,-1.0,-1.0,1.0,1.0,-1.0,0.0,...,-1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0
21537,1.0,1.0,1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,1.0,...,-1.0,0.0,0.0,1.0,-1.0,-1.0,-1.0,-1.0,0.0,1.0
71337,1.0,1.0,0.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,0.0,...,-1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0


In [ ]:
# Will  add the background charge here to cluster 1. Background charge is added in the file non_ms_crg.xlsx.
non_ms_crg = pd.read_excel("non_ms_crg.xlsx", index_col = 0)
non_ms_crg

In [ ]:
# create the dictionary from the pandas data frame
non_ms_crg_dict = dict(zip(non_ms_crg["non ms"], non_ms_crg.Charge))
non_ms_crg_dict

In [9]:
#Add charge column in each ms list
ms_cl1_all = ms_cl1
ms_cl1_all.append('Crg_cl1')

ms_cl2_all = ms_cl2
ms_cl2_all.append('Crg_cl2')

ms_co1_2_all = ms_co1_2
ms_co1_2_all.append('Crg_co1_2')

ms_cl3_all = ms_cl3
ms_cl3_all.append('Crg_cl3')

ms_cl4_all = ms_cl4
ms_cl4_all.append('Crg_cl4')

ms_co1_4_all = ms_co1_4
ms_co1_4_all.append('Crg_co1_4')


ms_cl5_all = ms_cl5
ms_cl5_all.append('Crg_cl5')

ms_cl6_all = ms_cl6
ms_cl6_all.append('Crg_cl6')


In [10]:
# make total charge and occupancy table  for cluster 1
cls1_unique_all = data1.groupby(ms_cl1_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
cls1_unique_all["Occupancy"] = (cls1_unique_all.Count/sum(cls1_unique_all.Count)).round(3)
cls1_unique = cls1_unique_all.T
cls1_unique["std"] = cls1_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
cls1_unique = cls1_unique.loc[cls1_unique['std'] != 0].T[:-1]
cls1_unique = cls1_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(cls1_unique))
cls1_unique

5


,HISE0038_cl1,ASPE0139_cl1,GLUQ0225_cl1,Crg_cl1,Count,Occupancy
3,1.0,-1.0,-1.0,0.0,5057190.0,0.581
0,0.0,-1.0,-1.0,-1.0,3203346.0,0.368
1,0.0,-1.0,0.0,0.0,343147.0,0.039
4,1.0,-1.0,0.0,1.0,90626.0,0.010
2,0.0,0.0,-1.0,0.0,5691.0,0.001


In [11]:
# make total charge and occupancy table  for cluster 2
cls2_unique_all = data1.groupby(ms_cl2_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
cls2_unique_all["Occupancy"] = (cls2_unique_all.Count/sum(cls2_unique_all.Count)).round(3)
cls2_unique = cls2_unique_all.T
cls2_unique["std"] = cls2_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
cls2_unique = cls2_unique.loc[cls2_unique['std'] != 0].T[:-1]
cls2_unique = cls2_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(cls2_unique))
cls2_unique

45


,GLUE0050_cl2,GLUE0051_cl2,GLUE0216_cl2,HISP0060_cl2,ASPQ0220_cl2,GLUQ0227_cl2,Crg_cl2,Count,Occupancy
2,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-2.0,3427701.0,0.394
6,-1.0,-1.0,-1.0,1.0,0.0,-1.0,-1.0,2320185.0,0.267
29,0.0,-1.0,-1.0,0.0,-1.0,-1.0,-2.0,896304.0,0.103
31,0.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,699078.0,0.080
32,0.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,439872.0,0.051
18,-1.0,0.0,-1.0,0.0,0.0,-1.0,-1.0,284091.0,0.033
4,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-2.0,118234.0,0.014
22,-1.0,0.0,-1.0,1.0,0.0,-1.0,0.0,73780.0,0.008
5,-1.0,-1.0,-1.0,1.0,-1.0,0.0,-1.0,67597.0,0.008
1,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-2.0,62893.0,0.007


In [12]:
# make total charge and occupancy table  for conn1-2
co1_2_unique_all = data1.groupby(ms_co1_2_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
co1_2_unique_all["Occupancy"] = (co1_2_unique_all.Count/sum(co1_2_unique_all.Count)).round(3)
co1_2_unique = co1_2_unique_all.T
co1_2_unique["std"] = co1_2_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
co1_2_unique = co1_2_unique.loc[co1_2_unique['std'] != 0].T[:-1]
co1_2_unique = co1_2_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(co1_2_unique))
co1_2_unique

2


,GLUE0388_co1_2,Crg_co1_2,Count,Occupancy
0,-1.0,-1.0,7522439.0,0.865
1,0.0,0.0,1177561.0,0.135


In [13]:
# make total charge and occupancy table  for cluster 3
cls3_unique_all = data1.groupby(ms_cl3_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
cls3_unique_all["Occupancy"] = (cls3_unique_all.Count/sum(cls3_unique_all.Count)).round(3)
cls3_unique = cls3_unique_all.T
cls3_unique["std"] = cls3_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
cls3_unique = cls3_unique.loc[cls3_unique['std'] != 0].T[:-1]
cls3_unique = cls3_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(cls3_unique))
cls3_unique

7


,HISE0058_cl3,GLUP0053_cl3,GLUQ0235_cl3,Crg_cl3,Count,Occupancy
3,1.0,-1.0,-1.0,-1.0,6235456.0,0.717
0,0.0,-1.0,-1.0,-2.0,1997564.0,0.230
4,1.0,-1.0,0.0,0.0,341600.0,0.039
1,0.0,-1.0,0.0,-1.0,108282.0,0.012
5,1.0,0.0,-1.0,0.0,12272.0,0.001
2,0.0,0.0,-1.0,-1.0,4221.0,0.000
6,1.0,0.0,0.0,1.0,605.0,0.000


In [14]:
# make total charge and occupancy table  for cluster 4
cls4_unique_all = data1.groupby(ms_cl4_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
cls4_unique_all["Occupancy"] = (cls4_unique_all.Count/sum(cls4_unique_all.Count)).round(3)
cls4_unique = cls4_unique_all.T
cls4_unique["std"] = cls4_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
cls4_unique = cls4_unique.loc[cls4_unique['std'] != 0].T[:-1]
cls4_unique = cls4_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(cls4_unique))
cls4_unique

44


,ASPP0072_cl4,GLUP0074_cl4,GLUP0110_cl4,GLUQ0130_cl4,GLUQ0163_cl4,GLUQ0213_cl4,HISQ0251_cl4,Crg_cl4,Count,Occupancy
31,0.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0,-2.0,6242226.0,0.717
34,0.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,-2.0,595938.0,0.068
35,0.0,0.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,333049.0,0.038
38,0.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-1.0,294826.0,0.034
9,-1.0,0.0,-1.0,0.0,-1.0,-1.0,1.0,-2.0,291877.0,0.034
37,0.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,-2.0,242617.0,0.028
16,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-3.0,199765.0,0.023
5,-1.0,0.0,-1.0,-1.0,0.0,-1.0,1.0,-2.0,121446.0,0.014
23,0.0,-1.0,-1.0,0.0,-1.0,-1.0,1.0,-2.0,88075.0,0.010
20,0.0,-1.0,-1.0,-1.0,0.0,-1.0,1.0,-2.0,81429.0,0.009


In [15]:
# make total charge and occupancy table  for conn1-4
co1_4_unique_all = data1.groupby(ms_co1_4_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
co1_4_unique_all["Occupancy"] = (co1_4_unique_all.Count/sum(co1_4_unique_all.Count)).round(3)
co1_4_unique = co1_4_unique_all.T
co1_4_unique["std"] = co1_4_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
co1_4_unique = co1_4_unique.loc[co1_4_unique['std'] != 0].T[:-1]
co1_4_unique = co1_4_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(co1_4_unique))
co1_4_unique

2


,HISQ0233_co1_4,Crg_co1_4,Count,Occupancy
0,0.0,-1.0,8155687.0,0.937
1,1.0,0.0,544313.0,0.063


In [16]:
# make total charge and occupancy table  for cluster 5
cls5_unique_all = data1.groupby(ms_cl5_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
cls5_unique_all["Occupancy"] = (cls5_unique_all.Count/sum(cls5_unique_all.Count)).round(3)
cls5_unique = cls5_unique_all.T
cls5_unique["std"] = cls5_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
cls5_unique = cls5_unique.loc[cls5_unique['std'] != 0].T[:-1]
cls5_unique = cls5_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(cls5_unique))
cls5_unique

2


,GLUS0032_cl5,Crg_cl5,Count,Occupancy
0,-1.0,-1.0,6280601.0,0.722
1,0.0,0.0,2419399.0,0.278


In [17]:
# make total charge and occupancy table  for cluster 6
cls6_unique_all = data1.groupby(ms_cl6_all).Count.sum().reset_index()
# lets make one column that has percentage occupancy
cls6_unique_all["Occupancy"] = (cls6_unique_all.Count/sum(cls6_unique_all.Count)).round(3)
cls6_unique = cls6_unique_all.T
cls6_unique["std"] = cls6_unique.std(axis = 1).round(3)

# # delete if std is zero and also remove std column
cls6_unique = cls6_unique.loc[cls6_unique['std'] != 0].T[:-1]
cls6_unique = cls6_unique.sort_values(by = 'Count', axis = 0, ascending = False)
print(len(cls6_unique))
cls6_unique

4


,LYSQ0190_cl6,GLUR0005_cl6,Crg_cl6,Count,Occupancy
3,1.0,0.0,-1.0,6397756.0,0.735
2,1.0,-1.0,-2.0,2280617.0,0.262
1,0.0,0.0,-2.0,15105.0,0.002
0,0.0,-1.0,-3.0,6522.0,0.001


In [18]:
# lets save all cluster unique charge
writer1 = pd.ExcelWriter('cluster_crg_unique_mq144_dry.xlsx', engine='xlsxwriter')
cls1_unique.to_excel(writer1, sheet_name='cls1_unique')
cls1_unique_all.to_excel(writer1, sheet_name='cls1_unique_all')
cls2_unique.to_excel(writer1, sheet_name='cls2_unique')
cls2_unique_all.to_excel(writer1, sheet_name='cls2_unique_all')
cls3_unique.to_excel(writer1, sheet_name='cls3_unique')
cls3_unique_all.to_excel(writer1, sheet_name='cls3_unique_all')
co1_2_unique.to_excel(writer1, sheet_name='co1_2_unique')
co1_2_unique_all.to_excel(writer1, sheet_name='co1_2_unique_all')
cls4_unique.to_excel(writer1, sheet_name='cls4_unique')
cls4_unique_all.to_excel(writer1, sheet_name= 'cls4_unique_all')
co1_4_unique.to_excel(writer1, sheet_name= 'co1_4_unique')
co1_4_unique_all.to_excel(writer1, sheet_name= 'co1_4_unique_all')
cls5_unique.to_excel(writer1, sheet_name='cls5_unique')
cls5_unique_all.to_excel(writer1, sheet_name='cls5_unique_all')
cls6_unique.to_excel(writer1, sheet_name='cls6_unique')
cls6_unique_all.to_excel(writer1, sheet_name='cls6_unique_all')
writer1.save()

In [ ]:
# lets try to find the covariance realtion in cluster 4
import pandas as pd
cl4_cov_data = pd.read_excel("cluster_crg_unique_mq144_dry.xlsx", sheet_name='cls4_unique', index_col = 0)
cl4_cov_data.head()

In [ ]:
df = cl4_cov_data
#remove the count
df1 = df.iloc[:, :-1]
df1.head()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
plt.figure(figsize=(10, 5))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df1.corr(), dtype=np.bool))
heatmap = sns.heatmap(df1.corr(), mask=mask, vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('MQ_fr144 Cluster 4', fontdict={'fontsize':18}, pad=16)
plt.savefig("heat_cls4_mq_fr144.pdf", dpi = 600, bbox_inches = "tight");


In [ ]:
plt.figure(figsize=(10, 5))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(df1.corr(), dtype=np.bool))
heatmap = sns.heatmap(df1.corr(), mask=mask, vmin=-0.4, vmax=0.3, annot=True, cmap='BrBG')
heatmap.set_title('MQ_fr144 Cluster 4', fontdict={'fontsize':18}, pad=16);


In [ ]:
# correlation for the cluster residues
import pandas as pd
data_all_cor = pd.read_csv("finalall_crg_cl_crg_count_order.csv",index_col = 0)
data_all_cor

In [ ]:
data_all_cor.T

In [ ]:
data_all_cor1 = data_all_cor
for x in data_all_cor1.columns:
    if x[9:] != "cl1" and  x[9:] != "cl2" and  x[9:] != "co1_2" and  x[9:] != "cl3" and  x[9:] != "cl4" and  x[9:] != "co1_4" and x[9:] != "cl5" and  x[9:] != "cl6":
        data_all_cor1.drop(columns = x, inplace = True)
data_all_cor1

In [ ]:
data_all_cor2 = data_all_cor1.T
data_all_cor2["std"] = data_all_cor2.std(axis = 1).round(3)
#data_all_cor3 = data_all_cor3.T
data_all_cor2

In [ ]:
data_all_cor3 = data_all_cor2.loc[data_all_cor2['std'] != 0].T[:-1]
data_all_cor3

In [ ]:
data_all_cor3.corr()

In [ ]:
# This is for all residues
data_all_cor4 = data_all_cor
data_all_cor4 = data_all_cor4.iloc[:,:-9]
data_all_cor5 = data_all_cor4.T
data_all_cor5["std"] = data_all_cor5.std(axis = 1).round(3)
data_all_cor6 = data_all_cor5.loc[data_all_cor5['std'] != 0].T[:-1]
data_all_cor6

In [ ]:
plt.figure(figsize=(40, 25))
mask = np.triu(np.ones_like(data_all_cor6.corr(), dtype=np.bool))
heatmap = sns.heatmap(data_all_cor6.corr(), mask=mask, vmin=-1.0, vmax=1.0, annot=True, cmap='BrBG')
heatmap.set_title('MQ_fr144', fontdict={'fontsize':18}, pad=16)
plt.savefig("heatmap_mq_fr144.png", dpi = 600, bbox_inches = "tight");

In [ ]:
# What I have done below is to find the most common microstate and then subtract it from each microstates in the dataframe.
# After the subtraction I have changed the value of any number different then zero to 1. 
# I do not care how much it changes but only if it has changed. 
# Then I have taken each microstate (row) in my dataframe and done an outer product to find a matrix representation of it.
# Added all of these together, normalized this matrix and done SVD to find which is the "strongest" eigenvector. 
# From this eigenvector we can see which residues are strongly interacting.



In [ ]:
df2 = df1
df2

In [ ]:
# most occurence
base_ms = df2.iloc[0]
base_ms

In [ ]:
diff_from_base = df1 - base_ms 
print(diff_from_base)

In [ ]:
plt.figure(figsize=(10, 5))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(diff_from_base.corr(), dtype=np.bool))
heatmap = sns.heatmap(diff_from_base.corr(), mask=mask, vmin=-0.4, vmax=0.4, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);
